# Проверка кода агента

Используйте ноутбук, чтобы по шагам запускать загрузку моков, выбор фреймворка и кодогенерацию.


In [1]:
from mas_automl.code_agent.load_mocks import load_mock_inputs

DATA_ANALYZE, METADATA, REGISTRY = load_mock_inputs()
print(f"Ключи data: {list[str](DATA_ANALYZE.keys())}")
print(f"Ключи metadata: {list(METADATA.keys())}")
print(f"Фреймворки: {list(REGISTRY.keys())}")


Ключи data: ['summary', 'priority', 'steps', 'example_pipeline_snippet', 'frameworks_recommended', 'rationale', 'estimated_complexity', 'confidence']
Ключи metadata: ['dataset_id', 'name', 'version', 'version_label', 'description', 'citation', 'paper_url', 'paper_reference', 'creator', 'contributor', 'collection_date', 'upload_date', 'language', 'licence', 'url', 'original_data_url', 'minio_url', 'format', 'file_id', 'default_target_attribute', 'ignore_attribute', 'row_id_attribute', 'num_rows', 'num_features', 'num_classes', 'num_missing_values', 'quality', 'dataset_type', 'tags', 'status', 'visibility', 'extra_info', 'local_path']
Фреймворки: ['AutoGluon', 'H2O AutoML', 'LightAutoML']


In [11]:
REGISTRY

{'AutoGluon': "AutoGluon — AutoML-фреймворк AWS для табличных, текстовых, визуальных и мультимодальных данных. Использует ансамблирование (stacking, bagging, blending) и автоматический подбор гиперпараметров. Архитектура модульная: AutoGluon-Tabular, AutoGluon-Multimodal, AutoGluon-TimeSeries. Поддерживает GPU, распределённое обучение и масштабирование на больших наборах данных. \n\nГде применять: универсальные задачи классификации и регрессии на табличных данных; задачи анализа изображений (classification, object detection) и текстов (sentiment analysis, NER) через AutoMM; смешанные типы данных (таблица + текст/изображение); Kaggle/продакшн-сценарии, где нужно получить максимум качества без ручной настройки. \n\nТехнические возможности: автоматическое кодирование категориальных признаков; генерация новых признаков; встроенный HPO (Bayesian Optimization + Random Search); сохранение и загрузка моделей; интеграция с PyTorch и MXNet; поддержка обучения на GPU. \n\nОграничения: плохо подхо

In [3]:
# DATA_ANALYZE, METADATA, REGISTRY
import pandas as pd

PATH_TO_CSV = METADATA.get('local_path')
dataset_df = pd.read_csv(PATH_TO_CSV)

dataset_df.info(), dataset_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   checking_status         1000 non-null   object 
 1   duration                1000 non-null   int64  
 2   credit_history          1000 non-null   object 
 3   purpose                 1000 non-null   object 
 4   credit_amount           1000 non-null   float64
 5   savings_status          1000 non-null   object 
 6   employment              1000 non-null   object 
 7   installment_commitment  1000 non-null   int64  
 8   personal_status         1000 non-null   object 
 9   other_parties           1000 non-null   object 
 10  residence_since         1000 non-null   int64  
 11  property_magnitude      1000 non-null   object 
 12  age                     1000 non-null   int64  
 13  other_payment_plans     1000 non-null   object 
 14  housing                 1000 non-null   o

(None,
   checking_status  duration                  credit_history   purpose  \
 0              <0         6  critical/other existing credit  radio/tv   
 1        0<=X<200        48                   existing paid  radio/tv   
 
    credit_amount    savings_status employment  installment_commitment  \
 0         1169.0  no known savings        >=7                       4   
 1         5951.0              <100     1<=X<4                       2   
 
       personal_status other_parties  ...  property_magnitude age  \
 0         male single          none  ...         real estate  67   
 1  female div/dep/mar          none  ...         real estate  22   
 
    other_payment_plans housing existing_credits      job num_dependents  \
 0                 none     own                2  skilled              1   
 1                 none     own                1  skilled              1   
 
    own_telephone foreign_worker class  
 0            yes            yes  good  
 1           none       

In [4]:
# --- Инициализация клиента ---
from mas_automl.code_agent.openai_wraper import LLMClient, LLMConfig

client = LLMClient(LLMConfig())

# --- Вызов выбора фреймворка ---


🔑 OPENAI_API_KEY = sk-or-v
🌐 BASE_URL = https://openrouter.ai/api/v1
KWARGS - {'model': 'qwen/qwen3-coder-30b-a3b-instruct', 'temperature': 0.0, 'openai_api_key': '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', 'openai_api_base': 'https://openrouter.ai/api/v1'}


C:\Users\User1\Desktop\ITMO_bootcamp\src\mas_automl\code_agent\openai_wraper.py:50: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import ChatOpenAI``.
  self._client = ChatOpenAI(**client_kwargs)


✅ ChatOpenAI клиент инициализирован для OpenRouter


In [5]:
client.chat('loh')

'Hello! It looks like you sent "loh" - is there something I can help you with? I\'m here to assist with questions, conversations, or any other support you might need. Feel free to let me know how I can help!'

## Кодогенерация

In [6]:
from mas_automl.code_agent.base_pipeline import choose_framework

framework, reason, prompt = choose_framework(DATA_ANALYZE, METADATA, REGISTRY, client)

# --- Вывод результата ---

print("🔧 Выбранный фреймворк:", framework)
print("📘 Обоснование:", reason)

Prompt - input_variables=['analysis_json', 'frameworks_list', 'metadata_json'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Ты — эксперт по AutoML и ML-инженер. Твоя задача — выбрать наиболее подходящий AutoML-фреймворк для табличных данных. Оцени качество данных, размер, тип задачи, ограничения и предложи лучший вариант из списка.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['analysis_json', 'frameworks_list', 'metadata_json'], input_types={}, partial_variables={}, template='Вот анализ датасета и метаданные:\n\n### 📊 Data Analysis\n{analysis_json}\n\n### 🧾 Metadata\n{metadata_json}\n\n### ⚙️ Доступные AutoML фреймворки\n{frameworks_list}\n\nПоясни свой выбор кратко, но содержательно. Если несколько подходят, выбери наиболее универсальный и стабильный вариант. Ответ верни в формате JSON:\n{{"framework": "...", "reason": ".

In [10]:
print(type(prompt))

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>


In [13]:
from mas_automl.code_agent.base_pipeline import generate_code, evaluate_code

code_sample = generate_code(framework, client, iteration=1, feedback="")
print(code_sample)
print(evaluate_code(code_sample, framework))


from autogluon.tabular import TabularPredictor

def train_model(train_df, test_df, label):
    predictor = TabularPredictor(label=label).fit(train_df)
    return predictor
(True, 'Проверки пройдены.')


In [12]:
from mas_automl.code_agent.base_pipeline import run_pipeline

result = run_pipeline(max_iterations=2, llm=client)
result


Prompt - input_variables=['analysis_json', 'frameworks_list', 'metadata_json'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Ты — эксперт по AutoML и ML-инженер. Твоя задача — выбрать наиболее подходящий AutoML-фреймворк для табличных данных. Оцени качество данных, размер, тип задачи, ограничения и предложи лучший вариант из списка.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['analysis_json', 'frameworks_list', 'metadata_json'], input_types={}, partial_variables={}, template='Вот анализ датасета и метаданные:\n\n### 📊 Data Analysis\n{analysis_json}\n\n### 🧾 Metadata\n{metadata_json}\n\n### ⚙️ Доступные AutoML фреймворки\n{frameworks_list}\n\nПоясни свой выбор кратко, но содержательно. Если несколько подходят, выбери наиболее универсальный и стабильный вариант. Ответ верни в формате JSON:\n{{"framework": "...", "reason": ".

ValueError: too many values to unpack (expected 2)